In [172]:
"""
Custom struct for topogrpahy functions
"""
struct Topography
    f::Function
    params::Tuple
end

"""
Function describing a flat plane
"""
function flat(x)
    0
end

"""
Function describing a bump
# Arguments
- `x`: x coordinate
- `w`: width scaling
- `c`: x coordinate for the center of the bump
- `h`: optional arg for height scaling
"""
function bump(x, w, c, h=1)
    if x <= c-w || x >= c+w
        return 0
    else
        return h * exp(-(w^2/(w^2-(x-c)^2)))
    end
end

"""
Function describing a rectangular mound
# Arguments
- `x`: x coordinate
- `c`: x coordinate for left hand corner of mound
- `d`: height of mound
- `delta`: steepness of mound
- `w`: width of mound
"""
function mound(x, c, h, delta, w)
    h * (1/pi * (atan((x-c)/delta) - atan((x-c-w)/delta)))
end

"""
Custom struct for initial condition functions
"""
struct InitialCondition
    f::Function
    params::Tuple
end

"""
Function describing a reflected sigmoid
# Arguments
- `x`: x coordinate
- `b`: precursor film height
- `xf`: center point of transition
- `c`: steepness of transition
- `h`: optional argument control initial height
"""
function reflected_sigmoid(x, b, xf, c, h=1)
    b + (h-b)/(1 + exp(1/c * (x - xf)))
end

reflected_sigmoid

In [ ]:
"""
Builds the system of ODEs (represented by du)
u = (h_-1, h_0, ..., h_nx, h_nx+1) where h_-1 and h_nx+1 are ghost points
"""
function system!(du, u, p, t)
    D, alpha, dx, topo, params = p

    function phi(i)
        u[i] + topo(dx*(i-2), params...)
    end
    function interp(i)
        1/2 * (u[i]^3 + u[i+1]^3)
    end
    function f1(i)
        1/(dx^2) * (interp(i-1)*(phi(i-1) - phi(i)) + interp(i)*(phi(i+1) - phi(i)))
    end
    function f2(i)
        (1/dx^4) * (interp(i-1)*(phi(i-2) - 3*phi(i-1) + 3*phi(i) - phi(i+1)) + interp(i)*(-phi(i-1) + 3*phi(i) - 3*phi(i+1) + phi(i+2)))
    end
    function f3(i)
        (1/(2*dx)) * (u[i+1]^3 - u[i-1]^3)
    end

    du[1] = 0
    du[2] = 0
    du[end-1] = 0
    du[end] = 0
    for i in 3:length(u)-2
        du[i] = D*cos(alpha)*f1(i) - f2(i) - sin(alpha)*f3(i)
    end
    return du
end

In [177]:
# Discretization of x-domain
nx = 800
Lx = 40
dx = Lx/nx
x = 0.0:dx:Lx

# Topography definition
bcenter = 20.0
bwidth = 7.0
topo = Topography(bump, (bwidth, bcenter))
#topo = Topography(flat, ())
s = topo.f.(x, topo.params...)

# Initial condition definition
b = 0.01
ic_center = 5
ic_steep = 0.5
ic_height = 0.1
ic_obj = InitialCondition(reflected_sigmoid, (b, ic_center, ic_steep, ic_height))
ic = ic_obj.f.(x, ic_obj.params...)
# Adding ghost points
pushfirst!(ic, ic[1])
push!(ic, ic[end])

# Parameters of the system of ODEs
D = 1
alpha = pi/2

xsym = Symbol.(x)
pushfirst!(xsym, Symbol("ghost1"))
push!(xsym, Symbol("ghost2"))

# Solve the system of ODEs
using DifferentialEquations

p = (D, alpha, dx, topo.f, topo.params)
tspan = (0.0, 2800.0); save_every = 5.0
f = ODEFunction(system!, syms=xsym)
prob = ODEProblem(f, ic, tspan, p, saveat=save_every)
sol = solve(prob, alg=Rodas4())

retcode: Success
Interpolation: 1st order linear
t: 561-element Vector{Float64}:
    0.0
    5.0
   10.0
   15.0
   20.0
   25.0
   30.0
   35.0
   40.0
   45.0
    ⋮
 2760.0
 2765.0
 2770.0
 2775.0
 2780.0
 2785.0
 2790.0
 2795.0
 2800.0
u: 561-element Vector{Vector{Float64}}:
 [0.09999591419181679, 0.09999591419181679, 0.09999548450517846, 0.09999500963277495, 0.09999448482343451, 0.09999390482653424, 0.09999326383952406, 0.09999255544994336, 0.0999917725713533, 0.09999090737254832  …  0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
 [0.09999591419181679, 0.09999591419181679, 0.09999587495167564, 0.09999579008409763, 0.09999565334289841, 0.09999545866201119, 0.09999520018193153, 0.09999487225600427, 0.099994469422593, 0.09999398633759836  …  0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
 [0.09999591419181679, 0.0999959141918168, 0.09999590630424526, 0.09999588457121897, 0.09999584306522741, 0.0999957759020393, 0.09999567725028274, 0.09999554133588082, 0.0999

In [178]:

using Plots, LaTeXStrings, ColorSchemes
Plots.PyPlotBackend()

plot_font = "Computer Modern"
default(fontfamily=plot_font, grid=false, color = ColorSchemes.berlin[1])

# Controls the time step between fluid profile plotting
time_skip = 50
dt = floor(Int64, time_skip/save_every)

plt = plot(x, s, dpi=200, xlabel=L"Dimensionless length $x$", ylabel=L"Dimensionless height $h$", legend=false, color="black",
        title=L"Fluid profile at $\delta t = %$time_skip$ intervals", ylims=(0, Inf), xlims=(0, Lx))
for i = 1:floor(Int64, tspan[2]/time_skip)+1
    plot!(plt, x, sol.u[1 + dt*(i-1)][2:end-1] + s)
end

# Animates fluid profile based on time steps sepcified in ODE solver
anim = @animate for i = 1:size(sol.u)[1]
    plot(x, s, color="black", xlabel=L"Dimensionless length $x$", ylabel=L"Dimensionless height $h$")
    plot!(x, sol.u[i][2:end-1]+s, legend=false, ylims=(0, 2), xlims=(0, Lx))
end

using Dates, CSV, DataFrames

run_tag = string(Dates.format(now(), "YYYYmmdd-HHMMSS"))
path = "../runs/$run_tag"

mkpath(path)
gif(anim, path * "/flow.gif", fps = 30)
savefig(plt, path * "/plt.png")
df = DataFrame(sol)
#=
CSV format
First Col - Timestamp
Rest of Cols - Data including ghost points
=#
CSV.write(path * "/data.csv", df)

function save_params(path, filename="params")
    open(path*"/"*filename*".txt", "w") do f
        domain_info = "# Domain Parameters\nnx: $nx\nLx: $Lx\ndx: $dx\n"
        topo_info = "\n# Topography Parameters\n$topo\n"
        init_cond_info = "\n# Initial Condition Parameters\n$ic_obj\n"
        ode_sys_info = "\n# ODE System Parameters\nD: $D\nalpha: $alpha\n"
        time_info = "\n# Time Parameters\ntime_span: $tspan\ntimestep_save: $save_every"
        write(f, domain_info * topo_info * init_cond_info * ode_sys_info * time_info)
    end
end

save_params(path)

┌ Info: Saved animation to 
│   fn = /Users/bsamineni/Documents/research_projects/acoustics_thin_film/notes/runs/20220225-214447/flow.gif
└ @ Plots /Users/bsamineni/.julia/packages/Plots/LI4FE/src/animation.jl:114


306